<a href="https://colab.research.google.com/github/ssingh1187/DS-Unit-2-Sprint-4-Model-Validation/blob/master/DS_Unit_2_Sprint_Challenge_4_Model_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Data Science Unit 2 Sprint Challenge 4 — Model Validation

Follow the instructions for each numbered part to earn a score of 2. See the bottom of the notebook for a list of ways you can earn a score of 3.

## Predicting Blood Donations

Our dataset is from a mobile blood donation vehicle in Taiwan. The Blood Transfusion Service Center drives to different universities and collects blood as part of a blood drive.

The goal is to predict the last column, whether the donor made a donation in March 2007, using information about each donor's history. We'll measure success using recall score as the model evaluation metric.

Good data-driven systems for tracking and predicting donations and supply needs can improve the entire supply chain, making sure that more patients get the blood transfusions they need.

#### Run this cell to load the data:

In [0]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data')

df = df.rename(columns={
    'Recency (months)': 'months_since_last_donation', 
    'Frequency (times)': 'number_of_donations', 
    'Monetary (c.c. blood)': 'total_volume_donated', 
    'Time (months)': 'months_since_first_donation', 
    'whether he/she donated blood in March 2007': 'made_donation_in_march_2007'
})

In [3]:
df.head()

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


## Part 1.1 — Begin with baselines

What **accuracy score** would you get here with a **"majority class baseline"?** 
 
(You don't need to split the data into train and test sets yet. You can answer this question either with a scikit-learn function or with a pandas function.)

In [4]:
df.isnull().sum()

months_since_last_donation     0
number_of_donations            0
total_volume_donated           0
months_since_first_donation    0
made_donation_in_march_2007    0
dtype: int64

In [6]:
df.columns

Index(['months_since_last_donation', 'number_of_donations',
       'total_volume_donated', 'months_since_first_donation',
       'made_donation_in_march_2007'],
      dtype='object')

In [7]:
df.head(1)

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007
0,2,50,12500,98,1


In [8]:
df.made_donation_in_march_2007.value_counts()

0    570
1    178
Name: made_donation_in_march_2007, dtype: int64

In [9]:
df.made_donation_in_march_2007.value_counts(normalize=True)

0    0.762032
1    0.237968
Name: made_donation_in_march_2007, dtype: float64

In [0]:
# To get accuracy score with a majority class baseline

y = df['made_donation_in_march_2007']
X = df.drop(columns = 'made_donation_in_march_2007')

majority_class = [0]
y_pred = majority_class * len(y)

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_pred)

0.7620320855614974

What **recall score** would you get here with a **majority class baseline?**

(You can answer this question either with a scikit-learn function or with no code, just your understanding of recall.)

In [29]:
from sklearn.metrics import recall_score
recall_score(y, y_pred)

# recall score is 0, which is expected since, recall  = true positive/actual positive,
# and majority class is 0 for baseline model

0.0

## Part 1.2 — Split data

In this Sprint Challenge, you will use "Cross-Validation with Independent Test Set" for your model evaluation protocol.

First, **split the data into `X_train, X_test, y_train, y_test`**, with random shuffle. (You can include 75% of the data in the train set, and hold out 25% for the test set.)


In [34]:
from sklearn.model_selection import train_test_split

X = df.drop(columns='made_donation_in_march_2007')
y = df['made_donation_in_march_2007']
  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((561, 4), (187, 4), (561,), (187,))

## Part 2.1 — Make a pipeline

Make a **pipeline** which includes:
- Preprocessing with any scikit-learn [**Scaler**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)
- Feature selection with **[`SelectKBest`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)([`f_classif`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html))**
- Classification with [**`LogisticRegression`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import f_classif, SelectKBest

pipeline = make_pipeline(
    RobustScaler(), 
    SelectKBest(f_classif), 
    LogisticRegression(solver='lbfgs', max_iter=2000))

## Part 2.2 — Do Grid Search Cross-Validation

Do [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) with your pipeline. Use **5 folds** and **recall score**.

Include these **parameters for your grid:**

#### `SelectKBest`
- `k : 1, 2, 3, 4`

#### `LogisticRegression`
- `class_weight : None, 'balanced'`
- `C : .0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0`


**Fit** on the appropriate data.

In [44]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'selectkbest__k': [1, 2, 3, 4],
    'logisticregression__class_weight': [None, 'balanced'],
    'logisticregression__C': [.0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0]
}

gridsearch = GridSearchCV(pipeline, param_grid, cv=5, scoring='recall',
                               verbose=5, return_train_score=True)
gridsearch.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1 
[CV]  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=4, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=4 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=4, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=4 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=4, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=4 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=4, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=balanced, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=balanced, selectkbest__k=1, score=0.7307692307692307, total=   0.0s
[CV] logisticregres

[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:    4.1s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('selectkbest', SelectKBest(k=10, score_func=<function f_classif at 0x7f02fd0d3510>)), ('logisticregression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_int...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'selectkbest__k': [1, 2, 3, 4], 'logisticregression__class_weight': [None, 'balanced'], 'logisticregression__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall', verbose=5)

## Part 3 — Show best score and parameters

Display your **best cross-validation score**, and the **best parameters** (the values of `k, class_weight, C`) from the grid search.

(You're not evaluated here on how good your score is, or which parameters you find. You're only evaluated on being able to display the information. There are several ways you can get the information, and any way is acceptable.)

In [49]:
results = pd.DataFrame(gridsearch.cv_results_)

results.sort_values(by='rank_test_score').head(1).T

,53
mean_fit_time,0.00648942
mean_score_time,0.00135961
mean_test_score,0.784519
mean_train_score,0.784615
param_logisticregression__C,100
param_logisticregression__class_weight,balanced
param_selectkbest__k,2
params,"{'logisticregression__C': 100.0, 'logisticregr..."
rank_test_score,1
split0_test_score,0.730769


####Stretch Goal #3

In [59]:
selector = gridsearch.best_estimator_.named_steps['selectkbest']
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print()
print('Features not selected:')
for name in unselected_names:
    print(name)

Features selected:
months_since_last_donation

Features not selected:
number_of_donations
total_volume_donated
months_since_first_donation


## Part 4 — Calculate classification metrics from a confusion matrix

Suppose this is the confusion matrix for your binary classification model:

<table>
  <tr>
    <th colspan="2" rowspan="2"></th>
    <th colspan="2">Predicted</th>
  </tr>
  <tr>
    <th>Negative</th>
    <th>Positive</th>
  </tr>
  <tr>
    <th rowspan="2">Actual</th>
    <th>Negative</th>
    <td>85</td>
    <td>58</td>
  </tr>
  <tr>
    <th>Positive</th>
    <td>8</td>
    <td>36</td>
  </tr>
</table>

In [0]:
true_negative  = 85
false_positive = 58
false_negative = 8
true_positive  = 36

actual_negative = 85 + 58
actual_positive = 8 + 36

predicted_negative = 85 + 8
predicted_positive = 58 + 36

Calculate accuracy

In [52]:
accuracy = ((true_negative + true_positive) / 
            (true_negative + false_positive + false_negative + true_positive))
print('Accuracy is:', accuracy)

Accuracy is: 0.6470588235294118


Calculate recall

In [53]:
recall = true_positive / actual_positive
print('Recall is:', recall)

Recall is: 0.8181818181818182


####Stretch Goal #4

In [55]:
precision = true_positive / predicted_positive
f1 = 2 * precision*recall / (precision+recall)
print('f1 score is:', f1)

f1 score is: 0.5217391304347826


In [56]:
false_positive_rate = false_positive/actual_negative
print('false positive rate is:', false_positive_rate)

false positive rate is: 0.40559440559440557


## BONUS — How you can earn a score of 3

### Part 1
Do feature engineering, to try improving your cross-validation score.

### Part 2
Add transformations in your pipeline and parameters in your grid, to try improving your cross-validation score.

### Part 3
Show names of selected features. Then do a final evaluation on the test set — what is the test score?

### Part 4
Calculate F1 score and False Positive Rate. 